In [6]:
#Import dependencies
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import gmaps
from pprint import pprint

from config import gmaps_api_key


In [55]:
state_coords = pd.read_csv('state_coords.csv')
state_coords.head()

,State,latitude,longitude,Name
0,AK,63.588753,-154.493062,Alaska
1,AL,32.318231,-86.902298,Alabama
2,AR,35.201050,-91.831833,Arkansas
3,AZ,34.048928,-111.093731,Arizona
4,CA,36.778261,-119.417932,California


In [63]:
parks_key = ""
parks_url = f"https://api.nps.gov/api/v1/parks?limit=600&fields=addresses&key={parks_key}"
response = requests.get(parks_url)
parks_data = response.json()

park_names = []
park_city = []
park_state = []
park_zip = []

for park in parks_data['data']:
    try: 
        park_city.append(park['addresses'][0]['city'])
        park_names.append(park['fullName'])
        park_state.append(park['addresses'][0]['stateCode'])
        park_zip.append(park['addresses'][0]['postalCode'])
    except IndexError:
        print("x")

df = {
    'Park name': park_names, 'Park city': park_city, 'Park state': park_state, 'Park zip': park_zip,
}

park_df = pd.DataFrame(df)
park_df.dropna(inplace=True)

park_df.head()


x
x
x
x


,Park name,Park city,Park state,Park zip
0,Abraham Lincoln Birthplace National Historical...,Hodgenville,KY,42748
1,Acadia National Park,Bar Harbor,ME,04609
2,Adams National Historical Park,Quincy,MA,02169
3,African American Civil War Memorial,Washington,DC,20001
4,African Burial Ground National Monument,New York,NY,10007


In [56]:
# park_df.groupby("Park state").count().reset_index().rename(columns={'State': 'Count'})

park_count = park_df.groupby('Park state').count().reset_index()
park_count.columns = ["State", "Parks", 'City','Zip']
del park_count['City']
del park_count['Zip']
park_count.head()



,State,Parks
0,AK,19
1,AL,7
2,AR,7
3,AS,1
4,AZ,21


In [48]:
park_state_coord = pd.merge(park_count, state_coords, on="State")
park_state_coord.head()

,State,Parks,latitude,longitude,Name
0,AK,19,63.588753,-154.493062,Alaska
1,AL,7,32.318231,-86.902298,Alabama
2,AR,7,35.201050,-91.831833,Arkansas
3,AZ,21,34.048928,-111.093731,Arizona
4,CA,30,36.778261,-119.417932,California


In [62]:
gmaps.configure(api_key='')
fig = gmaps.figure()
weights = park_state_coord['Parks'].astype(int)
locations = park_state_coord[["latitude", "longitude"]].astype(float)

heat_layer = gmaps.heatmap_layer(locations, weights=weights,point_radius=1, dissipating=False)
heat_layer.gradient = [(0,0,0,0), (0,255,255,.75), (0,0,255,.75),(0,0,128,.75)]
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [65]:
dance_df = pd.read_csv('dance.csv')
dance_df.head()

,Website,City,State,Weekday,Unnamed: 4,Role_name
0,http://www.thedancingbears.com/,Anchorage,AK,Fridays,20,
1,http://contraborealis.org/,Fairbanks,AK,Fridays and Saturdays,20,
2,http://www.juneaucontras.org/,Juneau,AK,Saturdays,5,
3,https://sites.google.com/site/bfootmad/,Birmingham,AL,Saturdays,12,
4,http://huntsvillecontra.dance,Huntsville,AL,Saturdays,27,


In [75]:
dance_count = dance_df.groupby('State').count().reset_index()
dance_count.columns = ["State", "Dances", 'Del','Weekday', 'Unnamed: 4', "Role_name"]
# del dance_count['State']
del dance_count['Unnamed: 4']
del dance_count['Role_name']
del dance_count['Del']
del dance_count['Weekday']

dance_count.head()

,State,Dances
0,AK,3
1,AL,2
2,AR,2
3,AZ,4
4,CA,29


In [77]:
park_dance = pd.merge(park_state_coord, dance_count, on="State")
park_dance.head()

,State,Parks,latitude,longitude,Name,Dances
0,AK,19,63.588753,-154.493062,Alaska,3
1,AL,7,32.318231,-86.902298,Alabama,2
2,AR,7,35.201050,-91.831833,Arkansas,2
3,AZ,21,34.048928,-111.093731,Arizona,4
4,CA,30,36.778261,-119.417932,California,29


In [81]:
gmaps.configure(api_key='')
fig = gmaps.figure()

#Parks
weights = park_state_coord['Parks'].astype(int)
locations = park_state_coord[["latitude", "longitude"]].astype(float)

#dances
dance_weights = park_dance['Dances'].astype(int)
dance_locations = park_dance[["latitude", "longitude"]].astype(float)

park_heat_layer = gmaps.heatmap_layer(locations, weights=weights,point_radius=1, dissipating=False)
park_heat_layer.gradient = [(0,0,0,0), (0,255,255,.75), (0,0,255,.75),(0,0,128,.75)]
dance_heat_layer = gmaps.heatmap_layer(dance_locations, weights=dance_weights,point_radius=1, dissipating=False)
dance_heat_layer.gradient = [(0,0,0,0), (255,0,0,.75), (128,0,128,.75),(255,0,255,.75)]

fig.add_layer(park_heat_layer)
fig.add_layer(dance_heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))